In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import os
import tensorflow as tf
import tensorflow_hub as hub

from keras.preprocessing.image import load_img , img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
# from keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

# **Displaying Images**

In [ ]:
picture_size = 48
folder_path = '../input/emotiondataset/data/'
# os.listdir(folder_path+"train/"+expression)  # gives the array of images    

In [ ]:
expression = 'angry'
plt.figure(figsize=(12,12))
for i in range(1,10,1):
    plt.subplot(3,3,i)
    img = load_img(folder_path+"train/"+expression+"/"+
                  os.listdir(folder_path+"train/"+expression)[i],
                   target_size=(picture_size,picture_size))
    plt.imshow(img)
plt.show()

# Making training and Validaton Data

In [ ]:
batch_size = 128 
datagen_train = ImageDataGenerator()
datagen_val = ImageDataGenerator()

train_set = datagen_train.flow_from_directory(folder_path+"train",
                                             target_size=(picture_size,picture_size),
                                             #color_mode = "grayscale",
                                             batch_size = batch_size,
                                             class_mode = 'categorical',
                                             shuffle=True)

test_set = datagen_val.flow_from_directory(folder_path+"test",
                                             target_size=(picture_size,picture_size),
                                              #color_mode = "grayscale",
                                              batch_size = batch_size,
                                             class_mode = 'categorical',
                                             shuffle=False)


In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(48,48,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

In [ ]:
#feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
#feature_extractor_model = "https://tfhub.dev/google/imagenet/mobilenet_v2_075_224/classification/5"

#pretrained_model_without_top_layer = hub.KerasLayer(
#    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
#base_model.trainable = False

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(7)

In [ ]:
model = tf.keras.Sequential([
  base_model,
  #pretrained_model_without_top_layer,
  global_average_layer,
  prediction_layer
])

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt,
             loss='categorical_crossentropy',
             metrics=["accuracy"])

model.summary()

# Fitting the Model with Training and Validation Data

In [ ]:
from keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau

checkpoint = ModelCheckpoint('./model.h5',
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             mode="max")

# early_stopping = EarlyStopping(monitor="val_loss",
#                               min_delta=0,
#                               patience=1,
#                               verbose=1,
#                               restore_best_weights=True)

reduce_learningrate = ReduceLROnPlateau(monitor="val_loss",
                                       factor=0.2,
                                       patience=3,
                                       verbose=1,
                                       min_delta=0.0001)

callbacks_list = [checkpoint,reduce_learningrate]
epochs = 50

In [ ]:
history = model.fit_generator(generator=train_set,
                              steps_per_epoch=train_set.n//train_set.batch_size,
                              epochs=epochs,
                              validation_data = test_set,
                              validation_steps = test_set.n//test_set.batch_size,
                              callbacks = callbacks_list)


In [ ]:
model.save('model.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()